In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

## Task1

빈 칸을 채워주세요!

단계별 output이 github 파일에는 남아있으니 그 output과 동일한 형태인지 확인하면서 진행해주시면 됩니다~

In [2]:
#1. 생성할 문장 데이터

sentence = ("Brick walls are there for a reason and you must not think "
            "that the brick walls aren't there to keep us out, but rather "
            "in this way that the brick walls are there to show us how badly we want things.")

In [3]:
#2. 문자 집합 만들기
world_set = list(set(sentence))

## 문제(1): 각 문자에 정수 인코딩 (공백도 하나의 원소로 포함)
vocab = {c: i for i, c in enumerate(world_set)}

In [4]:
print(vocab)

{'o': 0, 'm': 1, 'k': 2, 'f': 3, 'b': 4, 'd': 5, 'l': 6, 'r': 7, 'u': 8, 'i': 9, "'": 10, 'n': 11, 'g': 12, 'a': 13, 'B': 14, 'y': 15, ' ': 16, 'h': 17, 'p': 18, ',': 19, 'c': 20, 'e': 21, 'w': 22, 's': 23, 't': 24, '.': 25}


In [5]:
#3. 문자 집합 크기 확인

vocab_size = len(vocab)
print('문자 집합 크기 : {}'.format(vocab_size))

문자 집합 크기 : 26


In [7]:
#4. 하이퍼 파라미터 설정(자유롭게 수정해보세요!)

hidden_size = vocab_size # 같아야 하는 것 확인!
sequence_length = 11  
learning_rate = 0.01

In [8]:
#5. seqence 길이 단위 자르기

# 데이터 구성을 위한 리스트
x_data = []
y_data = []

## 문제(2): 반복문 내에서의 인덱싱을 사용하여 sequence_length 값 단위로 샘플을 잘라 데이터 만들기, y_str은 x_str은 한 칸씩 쉬프트된 sequnce

for i in range(0, len(sentence) - sequence_length):
  x_str = sentence[i:i + sequence_length]
  y_str = sentence[i + 1: i + sequence_length + 1]
  print(i, x_str, "->", y_str)

  # x_str과 y_str이 문자집합에 해당하는 인덱스를 각각 x_data, y_data에 append
  x_data.append([vocab[c] for c in x_str])
  y_data.append([vocab[d] for d in y_str])

0 Brick walls -> rick walls 
1 rick walls  -> ick walls a
2 ick walls a -> ck walls ar
3 ck walls ar -> k walls are
4 k walls are ->  walls are 
5  walls are  -> walls are t
6 walls are t -> alls are th
7 alls are th -> lls are the
8 lls are the -> ls are ther
9 ls are ther -> s are there
10 s are there ->  are there 
11  are there  -> are there f
12 are there f -> re there fo
13 re there fo -> e there for
14 e there for ->  there for 
15  there for  -> there for a
16 there for a -> here for a 
17 here for a  -> ere for a r
18 ere for a r -> re for a re
19 re for a re -> e for a rea
20 e for a rea ->  for a reas
21  for a reas -> for a reaso
22 for a reaso -> or a reason
23 or a reason -> r a reason 
24 r a reason  ->  a reason a
25  a reason a -> a reason an
26 a reason an ->  reason and
27  reason and -> reason and 
28 reason and  -> eason and y
29 eason and y -> ason and yo
30 ason and yo -> son and you
31 son and you -> on and you 
32 on and you  -> n and you m
33 n and you m ->  a

In [9]:
# 출력해서 한 칸씩 쉬프트된 것 확인하기!

print(x_data[0]) #Brick walls
print(y_data[0]) #rick walls ''(공백)

[14, 7, 9, 20, 2, 16, 22, 13, 6, 6, 23]
[7, 9, 20, 2, 16, 22, 13, 6, 6, 23, 16]


In [10]:
##6. 입력 시퀀스에 대해 원핫인코딩 수행

## 문제(4) : x_data를 원핫인코딩 > numpy의 eye를 쓸 수 있지 않을까?
x_one_hot = [np.eye(vocab_size)[x] for x in x_data]

##7. 입력 데이터, 레이블데이터 텐서로 변환

## 문제(5) : x_one_hot과 y_data 텐서로 변환 : 둘 다 같은 형식의 텐서로 변환하면 될까?? (FloatTensor, LongTesor 중 맞는 것은?)
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

#float tensor  > longtensor

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  if __name__ == '__main__':


In [11]:
##8. 크기 확인
print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([187, 11, 26])
레이블의 크기 : torch.Size([187, 11])


In [12]:
##9.원핫인코딩 결과 샘플 확인하기
print(X[0])

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0.,

In [13]:
##10. 레이블 데이터 샘플 확인하기
print(Y[0])

tensor([ 7,  9, 20,  2, 16, 22, 13,  6,  6, 23, 16])


In [14]:
##11. RNN 모델 구현

##문제(6) : 기본 pytorch 인자 넣기 연습 + forward 채우기
### 조건 : rnn layer 2개 쌓기 + 마지막은 fc layer
### batch_fisrt 설정 필요할까? (유튜브 강의 참고)

class Net(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, layers):
    super(Net, self).__init__()
    self.rnn = torch.nn.RNN(input_dim, hidden_dim, num_layers = layers, batch_first = True)
    self.fc = torch.nn.Linear(hidden_dim, hidden_dim, bias = True) 
  
  def forward(self, x):
    x, _status = self.rnn(x)
    x = self.fc(x)
    return x

In [16]:
net = Net(vocab_size, hidden_size, 2) #layer 2개 차곡차곡

In [20]:
##12. loss function
criterion = torch.nn.CrossEntropyLoss()

##13. optimizer
optimizer = optim.Adam(net.parameters(), learning_rate)

In [19]:
##14. 출력 크기 점검
outputs = net(X)
print(outputs.shape) #3차원

torch.Size([187, 11, 26])


In [21]:
##15. Training 시작

for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    ##문제(7) : outputs, Y 형태 그대로 넣으면 안되죠. view 함수를 이용해 loss값을 계산해봅시다.
    loss = criterion(outputs.view(-1, vocab_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    #16. 예측결과 확인
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오기
            predict_str += ''.join([world_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += world_set[result[-1]]

    print(predict_str)

               ,    ,        ,   ,                     ,                          ,       ,      ,,                     ,                                  ,    ,      ,                     y       
                                                                                                                                                                                                     
                                                                                                                                                                                                     
                                                                                                                                                                                                     
                                                                                                                                                                                                     
          

In [22]:
predict_str #오....

"rick walls are there tor a reason and you must not think that the brick walls are 't there to keep us out, but rather in this way that the brick walls are there to show us how badly we want thingsm"

결과가 어떤가요?? 마지막 에폭의 문장이 그럴싸한가요?

## Task2

위 sentence는 제가 임의로 생성한 문장들입니다.

마음에 드시는 문구 가져오셔서 문장이 어떻게 생성되는지 확인해보세요! 

영어가 아닌 한국어로 시도해보는 것도 좋겠죠? 

수정이 많이 필요(토큰화 등) 할 수 있으나 한번 시도해보시는 것 권장드립니다 :)

위 베이스라인은 어디든 수정하셔도 좋고 조금 더 자연스러운 문장이 나올 수 있게 다양한 시도를 해보세요!

조건 : 문장 3개 이상, 연결성이 있는 문장을 " " 으로 구분하여 ( )에 넣기

In [31]:
#1. 생성할 문장 데이터

kor_text = ("세상에 완벽한 문장은 없어, 완벽한 절망이 없듯이."
            "무라카미 하루키의 바람의 노래를 들어라에 나오는 문장이래."
            "이러니까 무라카미 하루키 책을 안 읽을 수가 없지.")

In [32]:
print(kor_text.split())

['세상에', '완벽한', '문장은', '없어,', '완벽한', '절망이', '없듯이.무라카미', '하루키의', '바람의', '노래를', '들어라에', '나오는', '문장이래.이러니까', '무라카미', '하루키', '책을', '안', '읽을', '수가', '없지.']


In [24]:
#https://wikidocs.net/64517
#한국어는 조사 때문에 형태소 토큰화를 진행한다...오
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 115, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 115 (delta 11), reused 10 (delta 3), pack-reused 91
Receiving objects: 100% (115/115), 1.27 MiB | 19.42 MiB/s, done.
Resolving deltas: 100% (50/50), done.
/content/Mecab-ko-for-Google-Colab
Installing konlpy.....
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 678 kB/s 
     |████████████████████████████████| 453 kB 72.4 MB/s 
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2022-08-16 08:00:53--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 18.205.93.1, 18.205.93.0, 18.205.93.2, ...
Connecting to bitbucket.org (bitbucket.org)|18.205.93.1|:443

In [35]:
from konlpy.tag import Mecab 
tokenizer = Mecab()
word = tokenizer.morphs(kor_text)

In [36]:
print(word)

['세상', '에', '완벽', '한', '문장', '은', '없', '어', ',', '완벽', '한', '절망', '이', '없', '듯이', '.', '무라카미', '하루키', '의', '바람', '의', '노래', '를', '들어라', '에', '나오', '는', '문장', '이', '래', '.', '이러', '니까', '무라카미', '하루키', '책', '을', '안', '읽', '을', '수', '가', '없', '지', '.']


In [39]:
#문자 집합 만들기
word_set = list(set(word)) 

##각 문자에 정수 인코딩
voca = {c: i for i, c in enumerate(word_set)}

In [38]:
print(voca)

{'에': 0, '세상': 1, '은': 2, '의': 3, '들어라': 4, '읽': 5, '이러': 6, '하루키': 7, '어': 8, '책': 9, '이': 10, '나오': 11, '는': 12, '듯이': 13, '문장': 14, '절망': 15, '가': 16, '바람': 17, '를': 18, '완벽': 19, '노래': 20, '수': 21, '지': 22, '래': 23, '니까': 24, '안': 25, '없': 26, '무라카미': 27, ',': 28, '을': 29, '한': 30, '.': 31}


In [40]:
#3. 문자 집합 크기 확인

voca_size = len(voca)
print('문자 집합 크기 : {}'.format(voca_size))

문자 집합 크기 : 32


In [42]:
#4. 하이퍼 파라미터 설정

hidden_size = voca_size 
sequence_length = 6 
learning_rate = 0.01

In [44]:
#5. seqence 길이 단위 자르기

# 데이터 구성을 위한 리스트
x_data = []
y_data = []


for i in range(0, len(word) - (sequence_length)):
    x_str = word[i:i+sequence_length]
    y_str = word[i+1:i+1+sequence_length] #sequence length를 유지해야한다.
    print(i, x_str, "->", y_str)

  # x_str과 y_str이 문자집합에 해당하는 인덱스를 각각 x_data, y_data에 append
    x_data.append([voca[c] for c in x_str])
    y_data.append([voca[d] for d in y_str])

0 ['세상', '에', '완벽', '한', '문장', '은'] -> ['에', '완벽', '한', '문장', '은', '없']
1 ['에', '완벽', '한', '문장', '은', '없'] -> ['완벽', '한', '문장', '은', '없', '어']
2 ['완벽', '한', '문장', '은', '없', '어'] -> ['한', '문장', '은', '없', '어', ',']
3 ['한', '문장', '은', '없', '어', ','] -> ['문장', '은', '없', '어', ',', '완벽']
4 ['문장', '은', '없', '어', ',', '완벽'] -> ['은', '없', '어', ',', '완벽', '한']
5 ['은', '없', '어', ',', '완벽', '한'] -> ['없', '어', ',', '완벽', '한', '절망']
6 ['없', '어', ',', '완벽', '한', '절망'] -> ['어', ',', '완벽', '한', '절망', '이']
7 ['어', ',', '완벽', '한', '절망', '이'] -> [',', '완벽', '한', '절망', '이', '없']
8 [',', '완벽', '한', '절망', '이', '없'] -> ['완벽', '한', '절망', '이', '없', '듯이']
9 ['완벽', '한', '절망', '이', '없', '듯이'] -> ['한', '절망', '이', '없', '듯이', '.']
10 ['한', '절망', '이', '없', '듯이', '.'] -> ['절망', '이', '없', '듯이', '.', '무라카미']
11 ['절망', '이', '없', '듯이', '.', '무라카미'] -> ['이', '없', '듯이', '.', '무라카미', '하루키']
12 ['이', '없', '듯이', '.', '무라카미', '하루키'] -> ['없', '듯이', '.', '무라카미', '하루키', '의']
13 ['없', '듯이', '.', '무라카미', '하루키', '의'] -> ['듯이', '.', '무

In [45]:
# 출력해서 한 칸씩 쉬프트된 것 확인하기!

print(x_data[0])
print(y_data[0])

[1, 0, 19, 30, 14, 2]
[0, 19, 30, 14, 2, 26]


In [46]:
##6. 입력 시퀀스에 대해 원핫인코딩 수행

## 문제(4) : x_data를 원핫인코딩 > numpy의 eye를 쓸 수 있지 않을까?
x_one_hot = [np.eye(voca_size)[x] for x in x_data]

##7. 입력 데이터, 레이블데이터 텐서로 변환

## 문제(5) : x_one_hot과 y_data 텐서로 변환 : 둘 다 같은 형식의 텐서로 변환하면 될까?? (FloatTensor, LongTesor 중 맞는 것은?)
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

#float tensor  > longtensor

In [47]:
##8. 크기 확인
print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([39, 6, 32])
레이블의 크기 : torch.Size([39, 6])


In [48]:
##9.원핫인코딩 결과 샘플 확인하기
print(X[0])

tensor([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])


In [49]:
##11. RNN 모델 구현

class Net(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, layers):
    super(Net, self).__init__()
    self.rnn = torch.nn.RNN(input_dim, hidden_dim, num_layers = layers, batch_first = True)
    self.fc = torch.nn.Linear(hidden_dim, hidden_dim, bias = True) 
  
  def forward(self, x):
    x, _status = self.rnn(x)
    x = self.fc(x)
    return x

In [50]:
net = Net(voca_size, hidden_size, 2) #layer 2개 차곡차곡

In [51]:
##12. loss function
criterion = torch.nn.CrossEntropyLoss()

##13. optimizer
optimizer = optim.Adam(net.parameters(), learning_rate)

In [52]:
##14. 출력 크기 점검
outputs = net(X)
print(outputs.shape) #3차원

torch.Size([39, 6, 32])


In [53]:
##15. Training 시작

for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    loss = criterion(outputs.view(-1, voca_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    #16. 예측결과 확인
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오기
            predict_str += ''.join([word_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += word_set[result[-1]]

    print(predict_str)

,책지,,,지,,,,,지지,,,,,지,,지,,지,,,,,지,,,,,지지지,,지,
,책들어라듯이,없수책책들어라없없없없없수세상,문장세상없수책없없책문장세상없없,책,세상책세상,한책없듯이없책,
없없없없없없없없없없없없없없없없하루키없없들어라없없없없없없문장없없없없하루키없하루키없없없없없없없없없없
없없없없없없없없하루키없없없없없없없하루키하루키없을없없없없없없문장하루키없없없하루키하루키하루키하루키없없없을없없없없없
없없없하루키없없.없하루키하루키없없없없.무라카미하루키하루키하루키을없무라카미없없없.문장하루키없.하루키하루키하루키하루키하루키을문장없을없없없없없
없없하루키하루키이없..하루키하루키을없없없.무라카미하루키의하루키을없무라카미없.없.문장하루키없.이러하루키하루키하루키하루키을을없을없없없..
없없하루키문장이없..하루키하루키이없없없.무라카미하루키의을을없을없.없.문장하루키없.이러하루키의하루키의을을이없없없없..
없.하루키문장이없..하루키하루키절망없없없.무라카미하루키의을을없를없없없.문장하루키없.이러하루키하루키하루키의을을이없없없없..
없.하루키문장이없..하루키하루키절망이없없.이러하루키의을의없를없에없는하루키하루키없.이러하루키하루키하루키의을을이이없없없..
없.하루키문장절망없.,하루키하루키절망이없없.이러하루키의을의이를없에없는하루키하루키래.이러하루키하루키하루키의을을이이없없없..
없.하루키문장절망없.,하루키한절망이없..이러하루키의을의노래를없에없는하루키의래.이러하루키하루키하루키의을을이절망없없없..
없.한문장이없.,하루키한절망이없..무라카미하루키의바람의노래를없에없는문장의없.이러하루키하루키하루키의을을이절망없없없..
완벽완벽한문장이없.,하루키한절망이없없.무라카미하루키의바람의노래를없를없는문장이없.이러하루키하루키하루키의을의이을가없없..
하루키완벽한문장이없.,하루키한절망이없없.무라카미하루키의바람의노래를없를없는문장이래.이러하루키무라카미하루키의을안이을가없없를.
무라카미완벽한문장이없.,하루키한절망가없없.무라카미하루키의바람의노래를없를없는문장이래.이러하루키무라카미하루키의을안읽을수가없를.
무라카미완벽한문장이없.,하루키한절망가없없.무라카미

In [54]:
predict_str

#결과가 너무 웃겨요
#한국어는 띄어쓰기 예측에 이슈가 있는 듯.

'에완벽한문장은없어,완벽한절망이없듯이.무라카미하루키의바람의노래를들어라에나오는문장이래.이러니까무라카미하루키책을안읽을수가없지.'

In [55]:
kor_text

'세상에 완벽한 문장은 없어, 완벽한 절망이 없듯이.무라카미 하루키의 바람의 노래를 들어라에 나오는 문장이래.이러니까 무라카미 하루키 책을 안 읽을 수가 없지.'